In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
from google.colab import drive

drive.mount('/content/drive/')
import time
from google.colab import files
from collections import deque
!pip install requests_cache
import requests_cache

Mounted at /content/drive/


In [ ]:
condition=True
while condition:
  try:
    city=input('Enter City Name \n')
    if 2<=len(city)<=50 and city.isalpha():
      condition=False
    else:
      print('Please input a valid City')
  except:
    continue


In [ ]:


# List contents of My Drive directory
drive_path = '/content/drive/My Drive/WebScraping Project' #can change the drive path as needed
os.listdir(drive_path)
# /content/drive/MyDrive/WebScraping Project/proxy_extract.py

In [ ]:
import sys
sys.path.append(f'{drive_path}')
from proxy_extract import _extract_proxies_free_proxy_list_net

In [ ]:
headers = {
    'authority': 'www.99acres.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': f'https://www.99acres.com/flats-in-{city}-ffid-page',
    'sec-ch-ua': '"Chromium";v="107", "Not;A=Brand";v="8"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'Expect':'',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/527.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

In [ ]:
requests_cache.install_cache(f'{drive_path}/my_cache', expire_after=3600)#Create a cache that will store the runtime information and you wont have to load anything even if your runtime ends

# **The main Script**

*   Handled Exceptions Correctly
*   Used sessions to clear cookies
*   Used Proxy Mesh to create multiple Proxies to reduce chances of IP ban
*   Handling Requests in such a way that if they throw exception, the loop will break but the page will restart from where they should be before the error
*   Use Placeholder to Handle Null data for raw as well as data structures
*   Used O(n) time complexity
*   Created a TimeOut such that if there is a delay in the response header, the script will handle it
*   The Mesh of proxies will refesh ones the whole pool is completed and the pool has currently 300 Proxies but i wont recommend it to use for sensitive data as it is open proxies which has security issues
*   Handled the HTTP header and exceptions such that if error occurs we know what caused the error



In [ ]:

import random
import requests
request=0
Placeholder=None

location=[]

All_Facilities=[]
Area=[]
Address=[]
NearbyLocations=[]
Description=[]
Features=[]
Builder_Name=[]
Building_Name=[]
Rating=[]
Property_id=[]
property_data =[]
location_advantages = []
project_details = {}



proxies=_extract_proxies_free_proxy_list_net(https=False)
P_300=[]
for proxy_obj in proxies:

    if proxy_obj['https']=='yes':
        proxy_obj['protocol']='https'
    else:
        proxy_obj['protocol']='http'
    proxy = f"{proxy_obj['protocol']}://{proxy_obj['ip']}:{proxy_obj['port']}"
    P_300.append(proxy)




# Function to extract proxies and create a queue with 9 repetitions for each proxy
def create_proxy_queue(proxies):
    proxy_queue = deque()
    for proxy in proxies:
        for _ in range(9):  # Repeat each proxy 9 times
            proxy_queue.append(proxy)
    return proxy_queue


# Create a queue of proxies with 9 repetitions for each proxy
proxy_queue = create_proxy_queue(P_300)

end=2589

# Load the value of j from the file if it exists this value will be k for next iteration
try:
    with open(f'{drive_path}/j_value.txt', 'r') as file:
        k = int(file.read().strip())
except FileNotFoundError:
    # Handle the case when the file doesn't exist (first run)
    k = 1  # Set a default value for k if needed





for j in range(k,end):
#     time.sleep(1)
    # Get the next proxy from the queue
    proxy = proxy_queue.popleft()
    session=requests.Session()
    links=[]

    try:
        url=f"https://www.99acres.com/flats-in-{city}-ffid-page-{j}"

        r=session.get(url,headers=headers,proxies={'http':proxy},timeout=5)
        r.raise_for_status()
        pageSoup = BeautifulSoup(r.content, 'html.parser')
        request+=1
        for page in pageSoup.find_all('div',class_="projectTuple__descCont"):
          try:
            links.append(page.a['href'])
          except:
            continue







        # Detail Page -Link start
        for link in links:

            try:
               page = session.get(link, headers=headers,proxies={'http':proxy},timeout=5)
               dpageSoup = BeautifulSoup(page.content, 'html.parser')
              #  time.sleep(random.uniform(1,3))
               request += 1
            except requests.exceptions.RequestException as e:
               print(f"An error occurred while making the request: {e}")
               continue

            property_info = {
                "Rating": Placeholder,
                "BHK Types": [],
                "Property Types": [],
                "Area Ranges (sq.ft.)": [],
                "Price Ranges (Cr)": [],
                "Additional Charges": [],
                "Construction Status" : [],
                "Completion Date":[]
            }



            # Rating by Features- Property Data
            try:
                rating = [i.text for i in dpageSoup.select_one('div.review__rightSide>div>ul>li>div').select('div.ratingByFeature__circleWrap')]
            except AttributeError:
                rating=None
            property_info["Rating"] = rating






            # Price Information BHK wise-Property Info->Property data
            all_id = ['1_1', '1_2', '1_3', '1_4', '1_5']
            for id in all_id:
                soup = dpageSoup.find('div', id=id)
                try:
                    bhk_type = soup.select_one('.configurationCards__configBandLabel').text.strip()
                except AttributeError:
                    bhk_type = Placeholder
                try:
                    property_type = soup.select_one('.configurationCards__configBandHeading').text.strip()
                except AttributeError:
                    property_type = Placeholder
                try:
                    area_range_text = soup.select_one('.configurationCards__cardAreaSubHeadingOne').text.strip()
                except AttributeError:
                    area_range_text = Placeholder
                try:
                    price_range_text = soup.select_one('.configurationCards__cardPriceHeading').text.strip()
                except AttributeError:
                    price_range_text = Placeholder
                try:
                    additional_charges_element = soup.select_one('.configurationCards__adtnCharges')
                    additional_charges = additional_charges_element.text.strip('+ ').strip() if additional_charges_element else Placeholder
                except AttributeError:
                    additional_charges = Placeholder

                property_info["BHK Types"].append(bhk_type)
                property_info["Property Types"].append(property_type)
                property_info["Area Ranges (sq.ft.)"].append(area_range_text)
                property_info["Price Ranges (Cr)"].append(price_range_text)
                property_info["Additional Charges"].append(additional_charges)





            # Construction Status-property info-> Property Data



            # Find the div with class ConstructionStatus__collapsedCard
            construction_status_div = dpageSoup.find('div', class_='ConstructionStatus__collapsedCard')

            # Initialize construction status and completion date with Placeholder
            construction_status = Placeholder
            completion_date = Placeholder

            try:
                # Extract construction status and completion date if the div is found
                construction_status_element = construction_status_div.select_one('.ConstructionStatus__phaseStatus')
                completion_date_element = construction_status_div.select_one('.ConstructionStatus__phaseStatusSubtitle')

                construction_status = construction_status_element.text.strip() if construction_status_element else Placeholder
                completion_date = completion_date_element.text.strip() if completion_date_element else Placeholder

            except Exception as e:
                # print(f"An error occurred while extracting construction status and completion date: {e}")
                pass

                # Continue with Placeholder values

            # Add construction status and completion date to the property_data dictionary
            property_info["Construction Status"].append(construction_status)
            property_info["Completion Date"].append(completion_date)


            property_data.append(property_info)





            #Facilities- All_Facilities
            try:
              Facility_info=dpageSoup.find_all('div',class_='UniquesFacilities__xidFacilitiesCard')
              Facilities=[i.text.strip() for i in Facility_info]
            except:
              Facilities=Placeholder
            All_Facilities.append(Facilities)

            #Builder Info

            # try:
            #   Builder_info=dpageSoup.find('div',class_="ProjectInfo__imgBox1 title_bold").text.strip()
            # except:
            #   Builder_info=Placeholder



            #Outer Details- location and Building_Name
            try:
                Outer_Details = dpageSoup.find('h1', class_='ProjectInfo__imgBox1 title_bold')
                location_text = Outer_Details.find('span', class_='ProjectInfo__hideTxt').text.strip()
                building_name = Outer_Details.contents[0].strip()
            except AttributeError:
                # Handle the exception if any of the elements are not found
                location_text = Placeholder
                building_name = Placeholder

            location.append(location_text)
            Building_Name.append(building_name)





            # Extracting project details - property_data
            try:
                project_details_div = dpageSoup.find('div', class_='AboutProjectDetail__specTable caption_strong_medium font_family')
                project_details_table = project_details_div.find('table', class_='AboutProjectDetail__specificTable')
                rows = project_details_table.find_all('tr')

                # Create a new dictionary for project details
                project_details = {key: Placeholder for key in ["Towers", "Floors", "Units", "Total Project Area", "Open Area"]}

                for row in rows:
                    columns = row.find_all('td')
                    if len(columns) == 2:
                        key = columns[0].text.strip()
                        value = columns[1].text.strip()
                        project_details[key] = value
            except AttributeError:
                # Handle the exception if the specified elements are not found
                project_details = {key: Placeholder for key in ["Towers", "Floors", "Units", "Total Project Area", "Open Area"]}

            # Append project details to the DataFrame
            property_data.append(project_details)





            #Builder Name- Builder_Name
            try:
              name=dpageSoup.find('div',class_='section_header_bold spacer4').text.strip()
            except:
              name=Placeholder
            Builder_Name.append(name)


            #Nearby Locations-location_advantages




          # Loop through each div tag and extract Location and Time
            for loc in dpageSoup.find_all('div', class_='locAdvantageCarousel__locAdBoxDesktop ellipsis'):
                try:
                    name = loc.find('div', class_='list_header_semiBold').text.strip()
                except AttributeError:
                    name = Placeholder  # Handle if name is not found
                try:
                    number = loc.find('div', class_='caption_subdued_medium').text.strip()
                except AttributeError:
                    number = Placeholder  # Handle if number is not found

                # Create a dictionary and append to the list
                location_dict = {
                    "Location_Near": name,
                    "Time_taken": number
                }
                location_advantages.append(location_dict)



            #Description-Description of each link
            try:
              Description.append(dpageSoup.find('div', {'data-label': 'PROJ_DESC'}).text.strip())
            except:
              Description.append(Placeholder)

        print(f'We are currently at: {j} page')

    except requests.exceptions.HTTPError as errh:
      if errh.response.status_code == 417:
        print(f"HTTP Error: {errh}")
        break

    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception: {err}")

     # If the proxy queue is empty, refill it for the next set of requests
    if not proxy_queue:
        proxy_queue = create_proxy_queue(proxies)




# Save the value of j to a file
with open(f'{drive_path}/j_value.txt', 'w') as file:
    file.write(str(j))


## Wrapping the whole code into csv which will be connected to the drive and concatinating the previous csv to the new csv, if previous Csv doesn't exist then create a new one and then append the items in that one





In [ ]:
All=[location,location_advantages,All_Facilities,Description,Builder_Name,Building_Name,property_data]

In [ ]:


import pandas as pd
import os

# Function to load existing data from CSV if it exists
def load_existing_data(csv_filepath):
    try:
        # Try to read the CSV file from the specified file path
        return pd.read_csv(csv_filepath)
    except FileNotFoundError:
      return pd.DataFrame(columns=["All Facilities", "Location", "Building Name", "Builder Name",
                                     "Location Advantages", "Description", "Property Data"])

# Load existing data from CSV into Main DataFrame

filepath = f'{drive_path}/data.csv'
Main = load_existing_data(filepath)

# List of lists to iterate through
lists_to_iterate = [All_Facilities, location, Building_Name, Builder_Name, location_advantages, Description, property_data]

# List of dictionaries to store property data
property_data_list = []

# Iterate through the lists and create property dictionaries
for data_points in zip(*lists_to_iterate):
    property_dict = {
        "All Facilities": data_points[0],
        "Location": data_points[1],
        "Building Name": data_points[2],
        "Builder Name": data_points[3],
        "Location Advantages": data_points[4],
        "Description": data_points[5],
        'Property Data': data_points[6]
    }
    property_data_list.append(property_dict)

# Create a DataFrame from the list of dictionaries
new_data = pd.DataFrame(property_data_list)

# Concatenate Main DataFrame and new_data vertically
Main = pd.concat([Main, new_data], ignore_index=True)

# Write the updated Main DataFrame to the CSV file
Main.to_csv(filepath, index=False)